This notebook calculates the doubling time of the T-cell lymphoblast population was estimated following a similar approach as in Li et al., 2020. This notebook also does Additional File 1 FigureS9 a and b with the trajectories of the relapse tumor growth and doubling time estimates. 

In [ ]:
import pandas as pd
import numpy as np
import json

from functools import reduce
from itertools import chain
from collections import OrderedDict

import seaborn as sns
import matplotlib.pyplot as plt

from statsmodels.discrete.discrete_model import Logit
import scipy.stats as st
from scipy.optimize import curve_fit,minimize

from aux_data_in_pyvar import config_rcparams

In [ ]:
config_rcparams()

In [ ]:
out_path = "" # path for the figures

In [ ]:
## FUNCTIONS

def get_points(df):
    
    dict_patients = {}
    dh = df.sort_values(by='days')
    for i in df.index:
        patient = df.loc[i, 'PATIENT']
        days = df.loc[i, 'days']
        blast_value = df.loc[i, 'blast ratio']
        blast_bool = int(blast_value > 0.01)  # pseudo-count
        blast_value = blast_value * blast_bool + 0.01 * (1 - blast_bool)
        dict_patients[patient] = dict_patients.get(patient, []) + [(days, blast_value)]
    return dict_patients


def sigmoid(x, a):
    
    return 1 / (1 + np.exp((-1) * a * x / 100))

def logsigmoid(x, a):
    
    return -np.log((1 + np.exp((-1) * a * x)))


def cross_entropy(patients, shifts, a):
    
    """
    patients : dict of point pairs per patient
    shifts   : dict of shifts per patients
    a        : growth parameter
    """
    
    score = 0
    
    for p in patients:
        
        y_initial = patients[p][0][1]
        t_initial = patients[p][0][0]
        y_final   = patients[p][1][1]
        t_final   = patients[p][1][0]
        
        s = shifts[p]
        
        score += y_initial * np.log(sigmoid(t_initial - s, a))
        score += (1 - y_initial) * np.log(1 - sigmoid(t_initial - s, a))
        score += y_final * np.log(sigmoid(t_final - s, a))
        score += (1 - y_final) * np.log(1 - sigmoid(t_final - s, a))
        
    return -score

def cost_function(params):
    
    a = params[0]
    shifts = {p: params[i + 1] for i, p in enumerate(set_patients)}
    return cross_entropy(dict_patients, shifts, a)

def partial_cost_function(a):
    """return another function letting the value of a fixed"""
    
    def func(shift_params):
        
        params = [a] + list(shift_params)
        return cost_function(params)
    
    return func


# setting the intial values for optimization
def generate_random_shifts():
    
    return np.array([-200. + np.random.choice(range(400), size=1)[0] for p in set_patients])

### Prepare data

In [ ]:
# get clinical data with days and pathologist blast measures
path_to_tsv = "" # table from Additional file 1 Table S1
df_clinical = pd.read_csv(path_to_tsv, sep='\t')
df_clinical.tail()

In [ ]:
# transform data
data = df_clinical[['Patient_id', 'Remission_sample_blasts', 'Relapse_sample_blasts','days_between_rem_rel']].dropna()
data['Remission_sample_blasts'] = data['Remission_sample_blasts'].apply(lambda x: float(x.replace("<", "")) if type(x) == str else float(x))
data['Relapse_sample_blasts'] = data['Relapse_sample_blasts'].apply(lambda x: float(x.replace(">", "")) if type(x) == str else float(x))

# PAT12 and PAT9 are clearly an outlier. Not used to fit the curve
data = data[data['Patient_id'] != 'PAT12']
data = data[data['Patient_id'] != 'PAT9']

data_rem = data[['Patient_id', 'Remission_sample_blasts']]
data_rem.rename(columns={'Remission_sample_blasts':"% blast", 'Patient_id':'PATIENT'}, inplace=True)
data_rem['days'] = 0
data_rel =  data[['Patient_id', 'Relapse_sample_blasts', 'days_between_rem_rel']]
data_rel.rename(columns={'Relapse_sample_blasts':"% blast", 'days_between_rem_rel':"days",'Patient_id':'PATIENT'}, inplace=True)

In [ ]:
data = pd.DataFrame()

data = data_rem.copy()
data = data.append(data_rel, ignore_index=True, sort=False)
data.to_csv("../intermediate_files/data_points_relapse.tsv", sep='\t', index=False)

### load data

In [ ]:
fn = '../intermediate_files/data_points_relapse.tsv'
df = pd.read_csv(fn, sep='\t')
df['blast ratio'] = df['% blast'].values / 100
df

In [ ]:
# dict_patients

dict_patients = get_points(df)
dict_patients

### Fitting growth rate and time-shift simultaneously

For each pair of data (time = initial or final) the initial estimate is not highly reliable. Consequently, by centering each pair first we are introducing quite a bit of error in the final fitting. 

Workaround: stadardize the time of the samples so that the initial and final estimates are well adjusted to some sigmoid of the form $\sigma(t, a) = (1 + e^{-at})^{-1}$ and find out the MLE estimate for $a$, i.e., find $\hat{a}$ and $S_i$ such that we minimize the cross-entropy:

$$\mathcal{L}(a;S_1,\ldots,S_N; y_1,\ldots, y_N) = \\ \sum_{i=1}^N C(t_{i, \texttt{initial}}, y_{i, \texttt{initial}}) + C(t_{i, \texttt{final}}, y_{i, \texttt{final}}) = \\
- \sum_{i=1}^N y_{i, \texttt{initial}} \log \left( \sigma(t_{i, \texttt{initial}} - S_i, a)\right) + (1 - y_{i, \texttt{initial}})\log\left(1 - \sigma(t_{i, \texttt{initial}} - S_i, a)\right) \\ 
- \sum_{i=1}^N y_{i, \texttt{final}} \log \left( \sigma(t_{i, \texttt{final}} - S_i, a)\right) + (1 - y_{i, \texttt{final}})\log\left(1 - \sigma(t_{i, \texttt{final}} - S_i, a)\right)$$

In [ ]:
set_patients = sorted(dict_patients.keys())
set_patients

In [ ]:
# optimize using a straightforward optimization approach

optimal_value = None
optimal_a = None
optimal_shifts = None

with open("../intermediate_files/optimization_values.txt", "a") as f:
    for _ in range(1000):

        initial_shift_params = generate_random_shifts()
        initial_params = [np.random.choice(range(10))] + list(initial_shift_params)
        res = minimize(cost_function, initial_params, options={'maxiter': 10000}, 
                       method='Nelder-Mead', tol=1e-10)
        value = cost_function(res.x)
        if (optimal_value is None) or (value < optimal_value):
            optimal_value = value
            optimal_a = res.x[0]
            optimal_shifts = res.x[1:]
            print(optimal_value, optimal_a, optimal_shifts)
            f.write("{} {} {}\n".format(optimal_value, optimal_a, optimal_shifts))

In [ ]:
x_array = []
y_array = []
patients = []
for i, p in enumerate(set_patients):
    for (x, y) in dict_patients[p]:
        x_array += [x - optimal_shifts[i]]
        y_array += [y]
        patients.append(p)
day = np.array(x_array)
blast = np.array(y_array)
plt.scatter(day, blast)
plt.show()

In [ ]:
# Logistic Regression again...

x = np.array([np.ones_like(day), day])
y = np.array(blast)
logit = Logit(y, x.T)
res = logit.fit()
res.summary()

In [ ]:
# estimators

intercept, slope = res.params

In [ ]:
probs = lambda arg: 1 / (1 + np.exp((-1) * (arg * slope + intercept)))
plt.scatter(day, blast)
x_vals = np.linspace(min(day), max(day), num=100)
y_vals = list(map(probs, x_vals))
plt.plot(x_vals, y_vals, c='r')
plt.hlines(0.5, min(day), max(day), linestyles='dashed')
plt.ylabel('blast ratio')
plt.xlabel('time')
plt.show()

### doing it with randomized data

In [ ]:
%%capture

params = []
day_list = []
blast_list = []
for _ in range(100):

    rand_index = np.random.choice(np.arange(len(day)), size=len(day), replace=True)
    day_rand = day[rand_index]
    blast_rand = blast[rand_index]
    day_list.append(list(day_rand))
    blast_list.append(list(blast_rand))
    x = np.array([np.ones_like(day_rand), day_rand])
    y = np.array(blast_rand)
    logit = Logit(y, x.T)
    res = logit.fit()
    params.append(res.params)

In [ ]:
# define color of points
dicc_colors = {'PAT10':'#d73027',
               'PAT11':'#f1b6da',
               'PAT13':'#fddbc7',
               'PAT14':'#80cdc1',
               'PAT1':'#d38d5fff',
               'PAT2':'#ffffbf',
               'PAT3':'#f46d43',
               'PAT4':'#66bd63',
               'PAT6':'#74add1',
               'PAT7':'#002255ff',
               'PAT8':'#aaaaffff',
               'PAT15':"#d4ff2aff",
               'PAT18':'#668000ff',
               'PAT19':'#d3bc5fff',
               'PAT16':'#782167ff'}

In [ ]:
#make figure of trajectories from bootstrap

fig, ax = plt.subplots(figsize=(8,4))

for i, (intercept, slope) in enumerate(params):
    probs = lambda arg: 1 / (1 + np.exp((-1) * (arg * slope + intercept)))
    x_vals = np.linspace(-400, 400, num=100)
    y_vals = list(map(probs, x_vals))
    ax.plot(x_vals, y_vals, c='#e0e0e0', alpha=0.05)

for i, pat in enumerate(patients):
    ax.scatter(day[i], blast[i], c=dicc_colors[pat], label=pat, edgecolor='#4d4d4d', s=50,
               zorder=3)
ax.set_ylabel("lymphoblast proportion")
ax.set_xlabel("days")
handles, labels = ax.get_legend_handles_labels()
handles_singles = [handles[j] for j in range(1,len(set_patients)*2+1,2)]
labels_singles = [labels[j] for j in range(1,len(set_patients)*2+1,2)]

plt.legend(handles=handles_singles,bbox_to_anchor=(0.4, 1),prop={'size': 10},ncol=2)
plt.title("Tumor growth trajectories in standardized time", fontsize=14)

plt.savefig(os.path.join(out_path,'fitting_logistic.png'), dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# get values of doubling time from bootstrap

values_doubling = list()

for intercept, alpha in params:
    values_doubling.append((np.log(2) / alpha))

In [ ]:
#make figure

fig, ax = plt.subplots(figsize=(2,4))

sns.boxplot(x=values_doubling, color='#e0e0e0', orient='v',ax=ax)

ax.text(x=0.1, y=round(np.percentile(values_doubling, 25), 2)-0.55, s=round(np.percentile(values_doubling, 25), 2))
ax.text(x=0.1, y=round(np.percentile(values_doubling, 75), 2)+0.1, s=round(np.percentile(values_doubling, 75), 2))
ax.text(x=0, y=round(np.mean(values_doubling),2)+0.05, s="mean:{}".format(round(np.mean(values_doubling),2)), ha='center')
ax.set_ylabel("doubling time")

for i,box in enumerate(ax.artists):
    box.set_edgecolor('black')
    box.set_linewidth(0.5)
    for j in range(i*5,i*5+5):
        line = ax.lines[j]
        line.set_color('black')
        line.set_mfc('black')
        line.set_mec('black')
        line.set_linewidth(0.9)
plt.title("Doubling time estimates \n from bootstrapping")
plt.savefig(os.path.join(out_path,'doubling_time_measure.png'), dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#write estimates in json for the following analysis (jupyter-nb )

data = {}
data['CI_upp'] = round(np.percentile(values_doubling, 75), 2) 
data['CI_down'] = round(np.percentile(values_doubling, 25), 2) 
data['mean'] = round(np.mean(values_doubling),2)
print(data)

with open('../intermediate_files/info_pop_cells.json', 'w') as fp:
    json.dump(data, fp)